In [1]:
import pickle
import numpy as np
import tensorflow as tf

import sys
sys.path.append('/Users/xiejunyi/Dropbox/Research/MultiLevelOptNN/util')
sys.dont_write_bytecode = True

In [2]:
with open('lidc_data.pkl', 'rb') as f:
    data = pickle.load(f)

print("data loaded")

data loaded


In [3]:
train_img = data[0][0]
print("training img:")
print(type(train_img), train_img.shape)

train_label = data[0][1]
print("training label:")
print(type(train_label), train_label.shape)

val_img = data[1][0]
print("validation img:")
print(type(val_img), val_img.shape)

val_label = data[1][1]
print("validation label:")
print(type(val_label), val_label.shape)

test_img = data[2][0]
print("test img:")
print(type(test_img), test_img.shape)

test_label = data[2][1]
print("test label:")
print(type(test_label), test_label.shape)


training img:
(<type 'numpy.ndarray'>, (22000, 4096))
training label:
(<type 'numpy.ndarray'>, (22000,))
validation img:
(<type 'numpy.ndarray'>, (1004, 4096))
validation label:
(<type 'numpy.ndarray'>, (1004,))
test img:
(<type 'numpy.ndarray'>, (953, 4096))
test label:
(<type 'numpy.ndarray'>, (953,))


In [4]:
# hyper parameters
learning_rate = 0.001
training_iters = 100000
batch_size = 128
display_step = 20

In [5]:
n_input = 4096
n_classes = 2
dropout = 0.75

In [6]:
# transform labels from number to one hot format
num_train = train_img.shape[0]
num_val = val_img.shape[0]
num_test = test_img.shape[0]

train_label_onehot = np.zeros([num_train, n_classes])
for idx in range(num_train):
    train_label_onehot[idx, train_label[idx]] = 1

val_label_onehot = np.zeros([num_val, n_classes])
for idx in range(num_val):
    val_label_onehot[idx, val_label[idx]] = 1
    
test_label_onehot = np.zeros([num_test, n_classes])
for idx in range(num_test):
    test_label_onehot[idx, test_label[idx]] = 1

In [7]:
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

In [8]:
def conv2d(img, w, b):
    return tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(img, w, strides=[1,1,1,1], padding='SAME'), b))

def max_pool(img, k):
    return tf.nn.max_pool(img, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

In [9]:
#train_img = tf.reshape(train_img, [-1, 64, 64])
#print('reshape input images and convert to tf.Tensor')
#print(type(train_img), train_img.get_shape())

# this function directly use default graph, later change to a function
# that uses an input graph
# decompose to tf basic function that has 'name' field
def conv_net(_X, _weights, _biases, _dropout):
    _X = tf.reshape(_X, shape=[-1,64, 64, 1])
    print(_X.get_shape())
    
    # Convolution Layer
    conv1 = conv2d(_X, _weights['wc1'], _biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = max_pool(conv1, k=2)
    # Apply Dropout
    conv1 = tf.nn.dropout(conv1, _dropout)
    print(conv1.get_shape())
    
    # Convolution Layer
    conv2 = conv2d(conv1, _weights['wc2'], _biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = max_pool(conv2, k=2)
    # Apply Dropout
    conv2 = tf.nn.dropout(conv2, _dropout)
    print(conv2.get_shape())
    
    # Fully connected layer
    # Reshape conv2 output to fit dense layer input
    dense1 = tf.reshape(conv2, [-1, _weights['wd1'].get_shape().as_list()[0]]) 
    # Relu activation
    dense1 = tf.nn.relu(tf.add(tf.matmul(dense1, _weights['wd1']), _biases['bd1']))
    # Apply Dropout
    dense1 = tf.nn.dropout(dense1, _dropout) # Apply Dropout
    print(dense1.get_shape())
    
    # Output, class prediction
    out = tf.add(tf.matmul(dense1, _weights['out']), _biases['out'])
    print(out.get_shape())
    return out

In [10]:
# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])), 
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])), 
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([16*16*64, 1024])), 
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, n_classes])) 
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [11]:
# Construct model
pred = conv_net(x, weights, biases, keep_prob)
#print(type(pred))

(?, 64, 64, 1)
(?, 32, 32, 32)
(?, 16, 16, 64)
(?, 1024)
(?, 2)


In [12]:
# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [13]:
# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [14]:
# Initializing the variables
init = tf.initialize_all_variables()

In [15]:
def next_batch(batch_size, input_samples, input_labels, n_classes, fake_data=False):
    index_in_epoch = 0
    num_examples = input_samples.shape[0]
    
    if fake_data:
        fake_image = [1.0 for _ in xrange(4096)]
        fake_label = 0
        return [fake_image for _ in xrange(batch_size)], [fake_label for _ in xrange(batch_size)]
    
    start = index_in_epoch
    index_in_epoch += batch_size
    
    if index_in_epoch > num_examples:
        # Finished epoch
        epochs_completed += 1
        # Shuffle the data
        perm = np.arange(num_examples)
        np.random.shuffle(perm)
        input_samples = input_samples[perm]
        input_labels = input_labels[perm]
        # start next epoch
        start = 0
        index_in_epoch = batch_size
        assert batch_size <= num_examples
    end = index_in_epoch
    output_samples = input_samples[start:end]

    #output_labels = np.zeros([end-start, n_classes])
    #original_labels = input_labels[start:end]
    #
    #for idx in range(batch_size):
    #    output_labels[idx, original_labels[idx]] = 1
    
    output_labels = input_labels[start:end]
    return output_samples, output_labels

In [1]:
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_xs, batch_ys = next_batch(batch_size, train_img, train_label_onehot, n_classes)
        
        #print(batch_xs.shape)
        #print(batch_ys.shape)
        sess.run(optimizer, feed_dict={x:batch_xs, y:batch_ys, keep_prob:dropout})
        if step % display_step == 0:
            # Calculate batch accuracy
            train_acc = sess.run(accuracy, feed_dict={x:batch_xs, y:batch_ys, keep_prob:1.})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_xs, y: batch_ys, keep_prob: 1.})
            print "Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                "{:.6f}".format(loss) + ", Training Accuracy= " + "{:.5f}".format(train_acc)
            
            val_acc = sess.run(accuracy, feed_dict={x: val_img, y: val_label_onehot, keep_prob:1.})
            print "Validation Accuracy= " + "{:.5f}".format(val_acc)
        step += 1
    print "Optimization Finished!"
    
    print "Testing Accuracy= ", sess.run(accuracy, feed_dict={x: test_img, y: test_label_onehot, keep_prob:1.})

NameError: name 'tf' is not defined